In [1]:
from zninit import ZnInit, Descriptor
import pathlib
import json

# Define a custom Descriptor which lazy-loads data from File
The following example is a minimal version of a lazy-loading implementation.
It is purely for illustration purpose. Lazy Loading is not a feature of `ZnInit` but can be relatively easily implemented with it.

In [2]:
class LazyData:
    """Identify Lazy Data"""


class LazyDataDescriptor(Descriptor):
    data_file = pathlib.Path("data.json")

    def load_from_file(self):
        """Read Data from json file"""
        data = json.loads(self.data_file.read_text())
        return data[self.name]

    def save_to_file(self, value):
        """Update json data file"""
        try:
            data = json.loads(self.data_file.read_text())
        except FileNotFoundError:
            data = {}
        data[self.name] = value
        self.data_file.write_text(json.dumps(data))

    def __get__(self, instance, owner=None):
        """reading the class attribute"""
        if instance is None:
            return self
        value = instance.__dict__.get(self.name, self.default)
        if value is LazyData:
            instance.__dict__[self.name] = self.load_from_file()

        return instance.__dict__.get(self.name, self.default)

    def __set__(self, instance, value):
        """writing the class attribute"""
        if value is LazyData:
            return
        self._instance = instance
        instance.__dict__[self.name] = value
        self.save_to_file(value)

Write a new `ZnInit` class, which defines two lazy descriptors. We have to disable `use_repr` because it will read the data from file automatically.

In [3]:
class SaveToFileData(ZnInit):
    name: str = LazyDataDescriptor(LazyData, use_repr=False)
    age: int = LazyDataDescriptor(LazyData, use_repr=False)

Instantiating a new instance will save the data into a `data.json` file.

In [4]:
data = SaveToFileData(name="Fabian", age=27)
print(data.__dict__)
data

{'age': 27, 'name': 'Fabian'}


SaveToFileData()

When we create a new instance of the class the `__dict__` will be empty. But when reading the attributes it will be filled by the data from `data.json`.

In [5]:
data = SaveToFileData()

In [6]:
data.__dict__

{}

In [7]:
data.name

'Fabian'

In [8]:
data.__dict__

{'name': 'Fabian'}

In [9]:
data.age

27

In [10]:
data.__dict__

{'name': 'Fabian', 'age': 27}